In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [2]:
df=pd.read_csv("minimum_temperature.csv")
df["Date"]=pd.to_datetime(df["Date"])
df.set_index('Date', inplace=True)
df

,Temp
Date,
1981-01-01,20.7
1981-01-02,17.9
1981-01-03,18.8
1981-01-04,14.6
1981-01-05,15.8
...,...
1990-12-27,14.0
1990-12-28,13.6
1990-12-29,13.5


In [3]:
df.dtypes

Temp    float64
dtype: object

In [4]:
train_size = int(len(df) * 0.8)
train, test = df.iloc[:train_size], df.iloc[train_size:]

In [5]:
scaler = MinMaxScaler(feature_range=(0, 1))
train_date = train.index
test_date = test.index

In [6]:
train_values = train.values.reshape(-1, 1)
test_values = test.values.reshape(-1, 1)

In [7]:
train_scaled = scaler.fit_transform(train_values)
test_scaled = scaler.transform(test_values)

In [8]:
train_scaled = train_scaled.reshape(-1, len(train.columns))
test_scaled = test_scaled.reshape(-1, len(test.columns))

In [9]:
train_scaled_df = pd.DataFrame(train_scaled, columns=train.columns, index=train_date)
test_scaled_df = pd.DataFrame(test_scaled, columns=test.columns, index=test_date)

In [10]:
def create_sequences(data, sequence_length):
    sequences = []
    for i in range(len(data) - sequence_length + 1):
        seq = data[i:i + sequence_length]
        sequences.append(seq)
    return np.array(sequences)

In [11]:
sequence_length = 10
X_train = create_sequences(train_scaled_df.values, sequence_length)
X_test = create_sequences(test_scaled_df.values, sequence_length)

In [12]:
X_train, y_train = X_train[:, :-1], X_train[:, -1][:, -1]
X_test, y_test = X_test[:, :-1], X_test[:, -1][:, -1]

In [13]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [14]:
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=2)

Epoch 1/50
91/91 - 2s - loss: 0.0352 - val_loss: 0.0102 - 2s/epoch - 17ms/step
Epoch 2/50
91/91 - 0s - loss: 0.0112 - val_loss: 0.0096 - 306ms/epoch - 3ms/step
Epoch 3/50
91/91 - 0s - loss: 0.0111 - val_loss: 0.0091 - 328ms/epoch - 4ms/step
Epoch 4/50
91/91 - 0s - loss: 0.0108 - val_loss: 0.0090 - 337ms/epoch - 4ms/step
Epoch 5/50
91/91 - 0s - loss: 0.0106 - val_loss: 0.0085 - 309ms/epoch - 3ms/step
Epoch 6/50
91/91 - 0s - loss: 0.0102 - val_loss: 0.0083 - 295ms/epoch - 3ms/step
Epoch 7/50
91/91 - 0s - loss: 0.0097 - val_loss: 0.0078 - 299ms/epoch - 3ms/step
Epoch 8/50
91/91 - 0s - loss: 0.0095 - val_loss: 0.0079 - 293ms/epoch - 3ms/step
Epoch 9/50
91/91 - 0s - loss: 0.0094 - val_loss: 0.0074 - 294ms/epoch - 3ms/step
Epoch 10/50
91/91 - 0s - loss: 0.0094 - val_loss: 0.0083 - 289ms/epoch - 3ms/step
Epoch 11/50
91/91 - 0s - loss: 0.0093 - val_loss: 0.0073 - 305ms/epoch - 3ms/step
Epoch 12/50
91/91 - 0s - loss: 0.0092 - val_loss: 0.0074 - 289ms/epoch - 3ms/step
Epoch 13/50
91/91 - 0s - lo

In [15]:
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')

23/23 [==============================] - 1s 6ms/step
Mean Squared Error: 0.007032928388177713
Mean Absolute Error: 0.06586242385638967
